## 数据预处理

In [37]:
import pandas as pd
df = pd.read_csv('data/tmp_test_data.csv', encoding='utf-8')
df.columns = df.columns.str.strip(' ')#去除列名中的空格
df.head()

用户ID                                               评论内容  会员级别  点赞数  \
0  1.000080e+11  黑色这款相当大气，*外形看起来舒服，双面玻璃，不锈钢机身，四周钢琴烤漆，握起来很有手感，也很...     0   19   
1  1.000080e+11  这个颜色很精致，特地选的，这次很给力五-佰的京东内 部折扣，可添迦Ⅴ:号：（181-7282...   103    2   
2  1.000080e+11  绿色的很漂亮，老婆喜欢。运行速度一流，显示效果不错，音质一流，拍照效果进步很大，尤其夜拍，很...     0  255   
3  1.000080e+11  外形看起来舒服，双面玻璃，不锈钢机身，四周钢琴烤漆，握起来很有手感，也很大气，手感不错，有份...   201  292   
4  1.000080e+11  手感特别好，颜色很喜欢，大小很合适，特别好看，屏幕音效也特别好，特别特别喜欢，拍照效果也特别...   201  485   

   回复数  得分    价格              购买时间  \
0   50   5  5999    2020/3/4 16:30   
1    3   5  5999    2020/3/6 13:31   
2  287   5  5999   2020/1/26 20:29   
3  198   5  5999    2020/1/5 10:54   
4  116   5  5999  2019/12/15 21:26   

                                               手机型号       销量  
0  Apple iPhone 11 (A2223) 128GB 黑色 移动联通电信4G手机 双卡双待  1377123  
1  Apple iPhone 11 (A2223) 128GB 黑色 移动联通电信4G手机 双卡双待  1377123  
2  Apple iPhone 11 (A2223) 128GB 黑色 移动联通电信4G手机 双卡双待  1377123  
3  Apple iPhone 11 (A2223) 128GB 黑色 移动联通电信4G手机 双卡双待  1377123  
4  Apple iPhone 11 (A2223) 128GB 黑色 移动联通电信4G手机 双卡双待  1377123

In [38]:
def format_phone_name(full_name,sub):
    full_name = str(full_name)
    index = full_name.find(sub)   #第一次出现的位置
    index2=full_name.find(sub,index+1)  #第二次出现的位置
    index3=full_name.find(sub,index2+1)  #第三次出现的位置
    new_name = full_name[:index3].translate(str.maketrans('/ \\','___'))
    return new_name

df = df.drop_duplicates()  #去重
#df.drop_duplicates(['评论内容','手机型号'])#若评论内容和手机型号一样，则认为数据无效，避免水军刷评论
df['评论内容'] = df['评论内容'].fillna('99999') #将空值所在行填充为99999 
df['手机型号'] = df['手机型号'].fillna('99999') 
row_comment_index = df[df.评论内容=='99999'].index.tolist()  #找出评论空值所在行索引
row_type_index = df[df.手机型号=='99999'].index.tolist()   #找出手机型号所在行索引
row_index = row_comment_index + row_type_index
df["手机型号"]=df.apply(lambda x:format_phone_name(x['手机型号'],' '),axis=1)
df = df.drop(row_index)
df.isnull().sum()  #查看每一列缺失值的数目
df.head()

用户ID                                               评论内容  会员级别  点赞数  \
0  1.000080e+11  黑色这款相当大气，*外形看起来舒服，双面玻璃，不锈钢机身，四周钢琴烤漆，握起来很有手感，也很...     0   19   
1  1.000080e+11  这个颜色很精致，特地选的，这次很给力五-佰的京东内 部折扣，可添迦Ⅴ:号：（181-7282...   103    2   
2  1.000080e+11  绿色的很漂亮，老婆喜欢。运行速度一流，显示效果不错，音质一流，拍照效果进步很大，尤其夜拍，很...     0  255   
3  1.000080e+11  外形看起来舒服，双面玻璃，不锈钢机身，四周钢琴烤漆，握起来很有手感，也很大气，手感不错，有份...   201  292   
4  1.000080e+11  手感特别好，颜色很喜欢，大小很合适，特别好看，屏幕音效也特别好，特别特别喜欢，拍照效果也特别...   201  485   

   回复数  得分    价格              购买时间             手机型号       销量  
0   50   5  5999    2020/3/4 16:30  Apple_iPhone_11  1377123  
1    3   5  5999    2020/3/6 13:31  Apple_iPhone_11  1377123  
2  287   5  5999   2020/1/26 20:29  Apple_iPhone_11  1377123  
3  198   5  5999    2020/1/5 10:54  Apple_iPhone_11  1377123  
4  116   5  5999  2019/12/15 21:26  Apple_iPhone_11  1377123

## 将同款手机评论整合到一起

In [39]:
import os
import shutil

path=r'input_comment'
cnt = 0
phone_name = ''

filenames = os.listdir(path)

substr = ' '
file_name = ''
phone_name = df.loc[0,'手机型号']
#df.loc[0,'手机型号'] = '1'
#存之前先清空文件夹
try:
    shutil.rmtree(path)
    print(path+"目录已经清空")
except:
    print(path+"目录不存在，进行创建")
os.mkdir(path)
# filenames = os.listdir(path)
file_name = ''
phone_name = df.loc[0,'手机型号']
try:
    for index,row in df.iterrows():
        if phone_name != row['手机型号']:
            df.loc[index,'手机型号'] = phone_name
            print(phone_name+"写入成功")
            phone_name = row['手机型号']
        file_name = path+os.sep+phone_name+'.txt'
        file = open(file_name, 'a', encoding='utf-8')
        file.write(row['评论内容'])
        file.close()
except Exception as e:
    print("手机评论写入文件出错，原因是： "+e)
df.head()

input_comment目录已经清空
Apple_iPhone_11写入成功
荣耀V30_PRO_李现同款写入成功
荣耀20青春版_AMOLED屏幕指纹_4000mAh大电池写入成功
荣耀9X_麒麟810_4000mAh续航写入成功
【超级爆款】华为_HUAWEI_P30写入成功
Apple_iPhone_XS写入成功
荣耀8X_千元屏霸_91%屏占比写入成功
荣耀20S_李现同款_3200万人像超级夜景写入成功
华为_HUAWEI_Mate写入成功
Redmi_Note8_4800万全场景四摄写入成功


用户ID                                               评论内容  会员级别  点赞数  \
0  1.000080e+11  黑色这款相当大气，*外形看起来舒服，双面玻璃，不锈钢机身，四周钢琴烤漆，握起来很有手感，也很...     0   19   
1  1.000080e+11  这个颜色很精致，特地选的，这次很给力五-佰的京东内 部折扣，可添迦Ⅴ:号：（181-7282...   103    2   
2  1.000080e+11  绿色的很漂亮，老婆喜欢。运行速度一流，显示效果不错，音质一流，拍照效果进步很大，尤其夜拍，很...     0  255   
3  1.000080e+11  外形看起来舒服，双面玻璃，不锈钢机身，四周钢琴烤漆，握起来很有手感，也很大气，手感不错，有份...   201  292   
4  1.000080e+11  手感特别好，颜色很喜欢，大小很合适，特别好看，屏幕音效也特别好，特别特别喜欢，拍照效果也特别...   201  485   

   回复数  得分    价格              购买时间             手机型号       销量  
0   50   5  5999    2020/3/4 16:30  Apple_iPhone_11  1377123  
1    3   5  5999    2020/3/6 13:31  Apple_iPhone_11  1377123  
2  287   5  5999   2020/1/26 20:29  Apple_iPhone_11  1377123  
3  198   5  5999    2020/1/5 10:54  Apple_iPhone_11  1377123  
4  116   5  5999  2019/12/15 21:26  Apple_iPhone_11  1377123

In [43]:
from collections import defaultdict
import os
import re
import jieba
import codecs
 
def seg_word(sentence):
    """使用jieba对文档分词"""
    seg_list = jieba.cut(sentence)
    seg_result = []
    for w in seg_list:
        seg_result.append(w)
    # 读取停用词文件
    stopwords = set()
    fr = codecs.open('motion_analysis/停用词.txt', 'r', 'utf-8')
    for word in fr:
        stopwords.add(word.strip())
    fr.close()
    # 去除停用词
    return list(filter(lambda x: x not in stopwords, seg_result))
 
def classify_words(word_dict):
    """词语分类,找出情感词、否定词、程度副词"""
    # 读取情感字典文件
    sen_file = open('motion_analysis//BosonNLP_sentiment_score//BosonNLP_sentiment_score.txt', 'r+', encoding='utf-8')
    # 获取字典文件内容
    sen_list = sen_file.readlines()
    # 创建情感字典
    sen_dict = defaultdict()
    # 读取字典文件每一行内容，将其转换为字典对象，key为情感词，value为对应的分值
    for s in sen_list:
        # 每一行内容根据空格分割，索引0是情感词，索引01是情感分值
        try:
            sen_dict[s.split(' ')[0]] = s.split(' ')[1]
        except:
            pass
        
 
    # 读取否定词文件
    not_word_file = open('motion_analysis//否定词.txt', 'r+', encoding='utf-8')
    # 由于否定词只有词，没有分值，使用list即可
    not_word_list = not_word_file.readlines()
 
    # 读取程度副词文件
    degree_file = open('motion_analysis//程度副词.txt', 'r+', encoding='utf-8')
    degree_list = degree_file.readlines()
    degree_dic = defaultdict()
    # 程度副词与情感词处理方式一样，转为程度副词字典对象，key为程度副词，value为对应的程度值
    for d in degree_list:
        try:
            degree_dic[d.split(',')[0]] = d.split(',')[1]
        except:
            pass
 
    # 分类结果，词语的index作为key,词语的分值作为value，否定词分值设为-1
    sen_word = dict()
    not_word = dict()
    degree_word = dict()
 
    # 分类
    for word in word_dict.keys():
        if word in sen_dict.keys() and word not in not_word_list and word not in degree_dic.keys():
            # 找出分词结果中在情感字典中的词
            sen_word[word_dict[word]] = sen_dict[word]
        elif word in not_word_list and word not in degree_dic.keys():
            # 分词结果中在否定词列表中的词
            not_word[word_dict[word]] = -1
        elif word in degree_dic.keys():
            # 分词结果中在程度副词中的词
            degree_word[word_dict[word]] = degree_dic[word]
    sen_file.close()
    degree_file.close()
    not_word_file.close()
    # 将分类结果返回
    return sen_word, not_word, degree_word
 
def list_to_dict(word_list):
    """将分词后的列表转为字典，key为单词，value为单词在列表中的索引，索引相当于词语在文档中出现的位置"""
    data = {}
    for x in range(0, len(word_list)):
        data[word_list[x]] = x
    return data
 
def get_init_weight(sen_word, not_word, degree_word):
    # 权重初始化为1
    W = 1
    # 将情感字典的key转为list
    sen_word_index_list = list(sen_word.keys())
    if len(sen_word_index_list) == 0:
        return W
    # 获取第一个情感词的下标，遍历从0到此位置之间的所有词，找出程度词和否定词
    for i in range(0, sen_word_index_list[0]):
        if i in not_word.keys():
            W *= -1
        elif i in degree_word.keys():
            # 更新权重，如果有程度副词，分值乘以程度副词的程度分值
            W *= float(degree_word[i])
    return W
 
def socre_sentiment(sen_word, not_word, degree_word, seg_result):
    """计算得分"""
    # 权重初始化为1
    W = 1
    score = 0
    # 情感词下标初始化
    sentiment_index = -1
    # 情感词的位置下标集合
    sentiment_index_list = list(sen_word.keys())
    # 遍历分词结果(遍历分词结果是为了定位两个情感词之间的程度副词和否定词)
    for i in range(0, len(seg_result)):
        # 如果是情感词（根据下标是否在情感词分类结果中判断）
        if i in sen_word.keys():
            # 权重*情感词得分
            score += W * float(sen_word[i])
            # 情感词下标加1，获取下一个情感词的位置
            sentiment_index += 1
            if sentiment_index < len(sentiment_index_list) - 1:
                # 判断当前的情感词与下一个情感词之间是否有程度副词或否定词
                for j in range(sentiment_index_list[sentiment_index], sentiment_index_list[sentiment_index + 1]):
                    # 更新权重，如果有否定词，取反
                    if j in not_word.keys():
                        W *= -1
                    elif j in degree_word.keys():
                        # 更新权重，如果有程度副词，分值乘以程度副词的程度分值
                        W *= float(degree_word[j])
        # 定位到下一个情感词
        if sentiment_index < len(sentiment_index_list) - 1:
            i = sentiment_index_list[sentiment_index + 1]
    return score
 
# 计算得分
def setiment_score(sententce):
    # 1.对文档分词
    seg_list = seg_word(sententce)
    # 2.将分词结果列表转为dic，然后找出情感词、否定词、程度副词
    sen_word, not_word, degree_word = classify_words(list_to_dict(seg_list))
    # 3.计算得分
    score = socre_sentiment(sen_word, not_word, degree_word, seg_list)
    return score

read_data = ''


# with open('input_comment//Apple iPhone 11.txt') as f:
#     read_data=f.read()
# 测试
# print(setiment_score(read_data))

In [46]:
print("开始计算得分")
content_path="input_comment"
filenames = os.listdir(content_path)
df = df.drop_duplicates(subset=['手机型号'],keep='first')#手机型号去重
df['评论得分'] = 0 #初始化自增列

for filename in filenames:
    phone_name = os.path.splitext(filename)[0]
    file_path = content_path+os.sep+filename
    with open(file_path,encoding='utf-8') as f:
        read_data=f.read()
        score = setiment_score(read_data)
        print('index:  '+str(df.index[df['手机型号'] == phone_name]))
        df.loc[df.index[df['手机型号'] == phone_name], '评论得分'] = score
    print(filename+"计算完毕")

tmp_df = df.drop(['评论内容','会员级别','点赞数','回复数','得分','购买时间'],axis=1)
print("处理过后的数据前五行如下：")
print(tmp_df.head())

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache


开始计算得分


Loading model cost 1.491 seconds.
Prefix dict has been built succesfully.


index:  Int64Index([0], dtype='int64')
Apple_iPhone_11.txt计算完毕
index:  Int64Index([4568], dtype='int64')
Apple_iPhone_XR.txt计算完毕
index:  Int64Index([2501], dtype='int64')
Apple_iPhone_XS.txt计算完毕
index:  Int64Index([4068], dtype='int64')
Redmi_Note8_4800万全场景四摄.txt计算完毕
index:  Int64Index([2001], dtype='int64')
【超级爆款】华为_HUAWEI_P30.txt计算完毕
index:  Int64Index([3568], dtype='int64')
华为_HUAWEI_Mate.txt计算完毕
index:  Int64Index([3068], dtype='int64')
荣耀20S_李现同款_3200万人像超级夜景.txt计算完毕
index:  Int64Index([1001], dtype='int64')
荣耀20青春版_AMOLED屏幕指纹_4000mAh大电池.txt计算完毕
index:  Int64Index([2874], dtype='int64')
荣耀8X_千元屏霸_91%屏占比.txt计算完毕
index:  Int64Index([1501], dtype='int64')
荣耀9X_麒麟810_4000mAh续航.txt计算完毕
index:  Int64Index([501], dtype='int64')
荣耀V30_PRO_李现同款.txt计算完毕
处理过后的数据前五行如下：
              用户ID    价格                           手机型号       销量         评论得分
0     1.000080e+11  5999                Apple_iPhone_11  1377123  1377.275364
501   1.000100e+11  3589                 荣耀V30_PRO_李现同款   106524  1371.7

# 数据清洗后存到数据库
读取爬虫脚本爬的CSV文件并清洗，随后存到MySQL中

## mysql建数据库指令
```mysql
mysql> create database jd_comment;
```
## mysql建表指令
```mysql
create table product_info(
    -> ID VARCHAR(100) NOT NULL,
    -> Name VARCHAR(200) NOT NULL,
    -> Price INT NOT NULL,
    -> Sales VARCHAR(50),
    -> CommentScore INT,
    -> PRIMARY KEY(ID)
    -> );
```

In [3]:
import pandas as pd
path = r"data//test_result.csv"
test_df = pd.read_csv(path,encoding='utf-8',header=0)
test_df.columns = test_df.columns.str.strip(' ')#去除列名中的空格

In [4]:
import pymysql
# 连接database
conn = pymysql.connect(host="localhost", user="root",password="")
# 得到一个可以执行SQL语句的光标对象
cursor = conn.cursor()
cursor

In [7]:
# 一个根据pandas自动识别type来设定table的type
def make_table_sql(df):
    df['用户ID'] = df['用户ID'].apply(str)
    columns = df.columns.tolist()
    types = df.ftypes
    # 添加id 制动递增主键模式
    make_table = []
    
    for item in columns:
        if 'int' in types[item]:
            char = item + ' INT'
        elif 'float' in types[item]:
            char = item + ' FLOAT'
        elif 'object' in types[item]:
            char = item + ' VARCHAR(255)'           
        elif 'datetime' in types[item]:
            char = item + ' DATETIME'            
        make_table.append(char)
    print(','.join(make_table))
    return ','.join(make_table)

# csv 格式输入 mysql 中
def csv2mysql(db_name, table_name, df):
    # 创建database
    cursor.execute('CREATE DATABASE IF NOT EXISTS {}'.format(db_name))
    # 选择连接database
    conn.select_db(db_name)
    # 创建table
    cursor.execute('DROP TABLE IF EXISTS {}'.format(table_name))
    cursor.execute('CREATE TABLE {}({})'.format(table_name,make_table_sql(df)))
    # 提取数据转list 这里有与pandas时间模式无法写入因此换成str 此时mysql上格式已经设置完成
    #df['日期'] = df['日期'].astype('str')
    values = df.values.tolist()
    # 根据columns个数
    s = ','.join(['%s' for _ in range(len(df.columns))])
    # executemany批量操作 插入数据 批量操作比逐个操作速度快很多
    insert_sql = 'INSERT INTO {} VALUES ({})'.format(table_name,s)
    cursor.executemany(insert_sql, values)
    conn.commit()
    conn.close()
    print("数据插入成功")
    
#导出到数据库
csv2mysql('jd_comment','product_info',test_df)
#导出到csv文件
#df.to_csv('data/result.csv',na_rep='NULL',sep = ',',index=False,header = False,encoding = 'utf_8_sig')